网络爬取之前的贮备工作
    检查robet.txt文件
    检查sitemap文件内容
    估算网站大小（渐变方法是通过google搜索site：+网络地址的方法来粗略确认网站的大小）
    识别网站使用的技术（builtwith.parse(网络主机地址)）可以显示网站的信息,见下面的例子
    寻找网站所有者（通过whois模块）
    

In [2]:
#设定固定的文件夹
in_path='D:/material/'
out_path='D:/result/'

In [2]:
#设定多个agent
import urllib.request as ul
import requests as rq
import urllib
from bs4 import BeautifulSoup as bl
import random
import re
auto_headers = ["Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0",
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36']

global headers
headers = {'User-Agent': random.choice(auto_headers)}

In [3]:
old_url = 'https://www.sosody.net/v/80806-2-0.html'
req = ul.Request(old_url)
response = ul.urlopen(req)
print ('Old_url:' + old_url)
print ('Real_url:' + response.geturl())

Old_url:https://www.sosody.net/v/80806-2-0.html
Real_url:https://www.sosody.net/v/80806-2-0.html


In [4]:
import builtwith
import whois
website='http://example.webscraping.com'
data=builtwith.parse(website)
for i in data:
    print(data[i])
print('\n\n\n')
data=whois.whois(website)
for i in data:
    print(data[i])

['Nginx']
['Web2py', 'Twitter Bootstrap']
['Python']
['jQuery', 'Modernizr', 'jQuery UI']






timeout: timed out

编写第一个网络爬虫

学习设置网络代理

In [10]:
import json
def download(website,num_retries=3):#增加自动重新下载的参数num_retries，默认数量为3,user_agent为设置代理
    global headers
    print("Downloading:",website)
    request=ul.Request(website,headers=headers)
    try:
        html=ul.urlopen(website).read()
    except ul.URLError as e:
        print("download Error:",e.reason)
        html=None
        if hasattr(e,'code') and 500<=e.code<=600:
            return(download(website,num_retries-1))
    return (html)
website='http://www.xbiquge.la/10/10489/9683462.html'
html=bl(download(website),'lxml')
infors=html.find_all('div',id_='content')
print(infors)

Downloading: http://www.xbiquge.la/10/10489/9683462.html
[]


学习使用遍历爬虫

In [ ]:
import itertools as it
for page in it.count(1):
    url='http://example.webscraping.com/view/-%d'% page
    html=download(url)
    if html is None:
        break
    else:
        print(url)
        print('\n')

Downloading: http://example.webscraping.com/view/-1
http://example.webscraping.com/view/-1


Downloading: http://example.webscraping.com/view/-2


In [ ]:
用来爬取链接的简单爬虫

In [ ]:
import re
def get_links(html):
    webpage_regex=re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
    return(webpage_regex.findall(html))

def link_crawler(seed_url,link_regex):
    crawl_queue=[seed_url]
    while crawl_queue:
        url=crawl_queue.pop()
        html=download(url)
        for link in get_links(html):
            if re.match(link_regex,link):
                crawl_queue.append(link)
    list=set(crawl_queue)
    print(list)

link_crawler("http://example.webscraping.com",'/(index/view)')

In [1]:
import sys

def ip_search(ip):
    host = 'http://saip.market.alicloudapi.com'
    path = '/ip'
    method = 'GET'
    appcode = '91fa754784494761b450672aba2d3efb'
    querys = 'ip='+ip
    bodys = {}
    url = host + path + '?' + querys
    head={'Authorization':'APPCODE '+appcode}
    request = ul.Request(url,headers=head)
    response = ul.urlopen(request)
    content = response.read()
    info=json.loads(content)
    return(info)
ip=input('请输入需要查询的IP地址:')
info=ip_search(ip)
k=info['showapi_res_body']
print('IP地址对应的信息如下:'+'\n')
for i in k:
    print(i,k[i])
c=input('请按任意键结束查询。。。。。。')

请输入需要查询的IP地址:192.168.0.1


NameError: name 'ul' is not defined

In [ ]:
url='https://www.mzitu.com/xinggan/page/2/'
global headers
req=rq.get(url,headers=headers)
html=req.text
#print(html)
uo_list=re.findall('<li><a(.*?)span></li>',html)
for i in uo_list:
    print(re.findall('https://www.mzitu.com/......',i)[0])

In [6]:
#制造首页地址的汇总
filename='pricepage.txt'
out_file=in_path+filename
count=1         
with open(out_file,'a') as tar:
    for i in range(1,51):
        link='http://www.100ppi.com/mprice/mlist-'+str(count)+'.html\n'
        tar.write(link)
        count+=1

In [9]:
linkname='link2.txt'
final_name=in_path+linkname
def get_pic_page(url):
    global headers
    req=rq.get(url,headers=headers,timeout=30)
    html=req.text
    uo_list=re.findall('<li><a(.*?)span></li>',html)
    with open(final_name,'a') as tar:
        count=1
        for i in uo_list:
            tar.write(re.findall('https://www.mzitu.com/......',i)[0]+'\n')
            count+=1
    

In [10]:
with open(out_file,'r') as sou:
    count=1
    for eachline in sou:
        get_pic_page(eachline.strip('\n'))
        count+=1
print(count)

ConnectionError: HTTPSConnectionPool(host='www.mzitu.com', port=443): Max retries exceeded with url: /xinggan/page/2/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000000007CC8AC8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [ ]:
#生意社价格数据爬虫
